In [1]:
import os
import concurrent.futures
import multiprocess as mp
import random


from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import lightkurve as lk
from astroquery.simbad import Simbad
import pandas as pd 
from numpy import genfromtxt


#tf
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
d = pd.read_csv('/Users/michaelsong/Documents/GitHub/starzam/Labels/unlabeled_stars.csv',delimiter=',').to_numpy()
print(d)
print(d.shape)

[[1027707 '19253846+3646103' nan ... nan 6 0]
 [1027900 '19254767+3644073' nan ... nan 8 0]
 [1160655 '19232193+3650379' nan ... nan 6 0]
 ...
 [12934848 '19201615+5221593' nan ... nan 7 0]
 [12935035 '19204068+5218589' nan ... nan 8 0]
 [12984227 '19201916+5227134' nan ... nan 6 0]]
(14693, 10)


In [3]:
test_lk = lk.search_lightcurve('KIC '+str(757137), author='Kepler',exptime=1800)[2].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
print(len(test_lk.time.value))
l = random.randrange(250, len(test_lk.time.value))
s = random.randrange(0,len(test_lk.time.value)-l)
print(s,l)
test_lk2 = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
test_periodogram = test_lk2.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
freq = np.asarray([test_periodogram.frequency,test_periodogram.power*100000])
print(freq[1])

4134
560 3115
[2.38717595 4.98240076 1.89745886 1.98573454 2.13904506 2.67009853
 7.68950466 3.21674043 1.41664814 3.49660421 2.61438879 2.42846368
 8.90297319 7.05706237 2.18517007 5.96455358 1.87591963 1.66184474
 2.24242271 4.8772712  1.53907568 2.95681911 2.08749862 1.22797766
 2.87227508 3.05120409 0.75569914 1.0308758  1.29267925 2.31700121
 1.25812043 1.08805494 1.11414924 0.47902996 1.24089141 0.81965095
 1.19930445 1.7230869  0.93413544 0.83128198 2.46015079 1.00693021
 1.4036626  0.55423869 0.87961735 0.75745757 1.02019455 0.03800585
 0.34414082 0.91614241 0.10966658 0.28447467 1.02887576 0.25831428
 0.17951299 0.2316026  0.47349301 0.61797327 0.67134244 0.93078689
 0.40901006 0.37436921 0.72774085 0.36650282 0.82994937 0.30218915
 0.12081216 0.52232936 0.10674265 0.85514989 0.39885947 0.41214387
 0.10306439 0.44754819 0.25503845 0.10641352 0.09482554 0.66335454
 0.16816507 0.29555178 0.34322167 0.19298871 0.26752673 0.52868875
 0.34218302 0.22609986 0.26763993 0.4119223  0.2

In [4]:
def prepsr(i,sr,j):
    r = False
    if r:
        test_lk = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
        l = random.randrange(240, len(test_lk.time.value))
        s = random.randrange(0,len(test_lk.time.value)-l)
        lightcurve = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
    else:
        lightcurve = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
    
    periodogram = lightcurve.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
    # periodogram = periodogram.bin(binsize=10, method='mean')
    freq = np.asarray([periodogram.frequency,periodogram.power*100000])
    # pd.DataFrame(freq[1]).to_csv("/content/drive/MyDrive/Research - Yihong Song and Rohit Prasanna/Yihong/Data/"+str(labeled_data["Kepler_ID"][i])+"_"+str(j)+".csv")
    return np.append(np.append(d[i][0:1],[j]),freq[1])

def getlc(i):
    lc_data = []
    sr = lk.search_lightcurve('KIC '+str(d[i][0]), author='Kepler',exptime=1800)
    # return prepsr(i,sr,1)
    for j in range(len(sr)):
        # x = wrap(prepsr(i,sr,j))
        # if x is not None:
        lc_data.append(prepsr(i,sr,j))
        # y = wrap(external_function_raises)
        # if y is not None:
        #     lc_data.append(y)
    return lc_data

def getdata(start):
    # for k in range(start,int(191449/100)+1,1):
    for k in range(start,start+1,1):
        lc_data = []
        for i in range(k*10,(k+1)*10,1):
            try:
                lc_data = lc_data+getlc(i)
            except:
                continue
            print("\r"+str(i),end="")
        # print(lc_data)
        pd.DataFrame(lc_data).to_csv("/Users/michaelsong/Documents/GitHub/starzam/Data_Unk/"+str(k)+".csv")

In [5]:
getdata(0)

9

In [9]:
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     print(executor.map(getdata, [1,2,3]))

with mp.Pool(8) as pool:
    print(pool.map(getdata, [x for x in range(1400,1470)]))

14659[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
